In [8]:
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

class GPT3Classifier(nn.Module):
    def __init__(self, base_model, num_labels):
        super(GPT3Classifier, self).__init__()
        self.gpt3 = base_model
        self.classifier = nn.Linear(self.gpt3.config.n_embd, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.gpt3(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state[:, -1, :]  # 取最后一个token的输出
        logits = self.classifier(last_hidden_state)
        return logits

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained("Irina/fantasy_GPT3Medium")
base_model = AutoModelForCausalLM.from_pretrained("Irina/fantasy_GPT3Medium")
base_model.config.pad_token_id = tokenizer.eos_token_id

# 创建分类模型实例
num_labels = 2  # 假设有两个类别
model = GPT3Classifier(base_model, num_labels)

# 假设你已经有了处理好的数据集
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 示例数据
texts = ["这是一个例子。", "这是一个更长的例子文本。"]
labels = [0, 1]  # 假设的标签

# 创建数据集和数据加载器
dataset = TextDataset(texts, labels)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# 训练模型（简化示例）
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
model.train()
for epoch in range(3):  # 训练3个epoch
    for batch in train_loader:
        inputs, labels = batch['input_ids'], batch['labels']
        attention_mask = batch['attention_mask']
        logits = model(inputs, attention_mask)
        loss = nn.CrossEntropyLoss()(logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# 注意：这只是一个示例，实际应用中需要更完整的数据处理和训练过程。

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [3]:
import os
import torch
from tqdm.auto import tqdm
import warnings
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd

# 忽略特定的警告
warnings.filterwarnings("ignore", message="Be aware, overflowing tokens are not returned*")


In [4]:
# 加载数据
new_file_path = './dataset/Mozilla_bug_raw_processed.csv'
df = pd.read_csv(new_file_path, encoding='latin-1')
df = df[['bug_id', 'summary', 'who', 'description']]
label_dict = {label: idx for idx, label in enumerate(df['who'].unique())}
df['label'] = df['who'].replace(label_dict)

# 合并bug_id和summary作为模型的输入
df['text_input'] =df['description'].astype(str)  # 使用空格作为分隔符
X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.label.values, test_size=0.15, random_state=42, stratify=df.label.values)
df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10572\494898259.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['who'].replace(label_dict)


In [5]:

# 对训练和验证数据的合并文本进行编码
# 对训练和验证数据的合并文本进行编码
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text_input.tolist(),  # 使用合并后的文本，并转换为列表
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',  # 更新pad_to_max_length为padding
    max_length=512, 
    truncation=True,  # 明确启用截断
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text_input.tolist(),  # 使用合并后的文本，并转换为列表
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',  # 更新pad_to_max_length为padding
    max_length=512, 
    truncation=True,  # 明确启用截断
    return_tensors='pt'
)

# 准备Tensor数据
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

# 定义DataLoader
batch_size = 1
train_loader = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
val_loader = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=2)

In [6]:
# # 初始化gpt2模型
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_dict), output_attentions=False, output_hidden_states=False)
# from transformers import GPT2Tokenizer, GPT2ForSequenceClassification,GPT2Config
# # 感谢外国诱人stackoverflow帮我解决的bug
# # instantiate the configuration for your model, this can be imported from transformers
# configuration = GPT2Config()
# # set up your tokenizer, just like you described, and set the pad token
# GPT2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# # instantiate the model
# model_name="gpt2"

# model = GPT2ForSequenceClassification(configuration).from_pretrained(model_name).to(device)
# # set the pad token of the model's configuration
# model.config.pad_token_id = model.config.eos_token_id
# model.resize_token_embeddings(len(tokenizer))

# optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)


In [7]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config, AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 确保这里使用的是正确的分词器变量名
GPT2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# 设置pad_token为eos_token
GPT2_tokenizer.pad_token = GPT2_tokenizer.eos_token

# 通过GPT2Config设置num_labels参数
num_labels = len(label_dict)  # 请替换为实际的标签数量
configuration = GPT2Config.from_pretrained("gpt2", num_labels=num_labels)

model = GPT2ForSequenceClassification.from_pretrained("Irina/fantasy_GPT3Medium", config=configuration).to(device)
# 设置模型的pad_token_id
#感谢stackoverflow
model.config.pad_token_id = GPT2_tokenizer.pad_token_id
# 使用GPT2_tokenizer的长度来调整模型的token embeddings大小
model.resize_token_embeddings(len(GPT2_tokenizer))

optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

KeyboardInterrupt: 

In [ ]:
# import pandas as pd
# from transformers import BertTokenizer

# # 加载新的数据集
# new_file_path = './data_开发者阈值10_词频阈值10/Mozilla_total_10_10.csv'
# columns_to_extract = ['bug_id', 'product', 'abstracts', 'description', 'component', 'severity', 'priority', 'history', 'status']
# df = pd.read_csv(new_file_path, usecols=columns_to_extract, encoding='latin-1')

# # 合并文本信息为模型的输入，除了developer列
# df['text_input'] = df[['bug_id', 'product', 'abstracts', 'description', 'component', 'severity', 'priority', 'history', 'status']].astype(str).agg(' '.join, axis=1)

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# # 对所有合并后的文本进行编码
# encoded_inputs = tokenizer.batch_encode_plus(df['text_input'].tolist(), add_special_tokens=True, truncation=True, padding=False, max_length=512)

# # 计算所有编码后的长度
# lengths = [len(input_ids) for input_ids in encoded_inputs['input_ids']]

# # 计算平均长度、中位数、最大和最小长度
# average_length = sum(lengths) / len(lengths)
# median_length = sorted(lengths)[len(lengths) // 2]
# max_length = max(lengths)
# min_length = min(lengths)

# print(f"Average length: {average_length}")
# print(f"Median length: {median_length}")
# print(f"Max length: {max_length}")
# print(f"Min length: {min_length}")

In [ ]:

model.to(device)

checkpoint_path = 'model_checkpoint_GPT2_Mozilla_bug_raw_processed_filtered.pth'

# 检查是否有可用的检查点
if os.path.isfile(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f'Resuming training from epoch {start_epoch}')
else:
    start_epoch = 0
    print('Starting training from scratch')



Starting training from scratch


In [ ]:
import pymysql
from datetime import datetime

# 数据库连接信息
host = '38.147.173.234'
user = 'lijianye'
password = '660013'
db = 'training_statistics_db'
experiment_num = 8
# 模型名称，根据实际情况手动设置
model_name = 'gpt2'
# 学习率和可选特性，根据实际情况手动设置
learning_rate = 1e-5  # 示例学习率
optional_feature = 'descrition'  # 示例可选特性
dataset = new_file_path
num_epochs = 10
for epoch in range(start_epoch, num_epochs):
    model.train()
    start_time = datetime.now()
        
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix(loss=loss.item())
    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, checkpoint_path)
    model.eval()
    correct_topk = {k: 0 for k in range(1, 11)}
    total = 0
    val_progress_bar = tqdm(val_loader, desc="Validating")
    
    for batch in val_progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        total += labels.size(0)
        
        # 计算top1到top10的正确率
        _, predicted_topk = torch.topk(logits, k=10, dim=1)
        labels_expanded = labels.unsqueeze(1)
        for k in range(1, 11):
            correct_topk[k] += (predicted_topk[:, :k] == labels_expanded).any(dim=1).sum().item()
                
    # 打印每个topK的准确率
    top10accuracy = []  # 初始化存储Top1到Top10准确率的数组

    for k in range(1, 11):
        accuracy = 100 * correct_topk[k] / total
        top10accuracy.append(accuracy)  # 将计算出的准确率添加到数组中
        print(f'Accuracy after epoch {epoch + 1}: Top{k}: {accuracy:.2f}%')
        print(top10accuracy)
    import pandas as pd
    import os
        # ...训练结束时间
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()/60.0
    # 定义数据字典，用于创建DataFrame
    data = {
            'epoch': [epoch],
            'start_time': [start_time],
            'end_time': [end_time],
            'duration': [duration],
            'user_id': [1],
            'model': [model_name],
            'top1_accuracy': [top10accuracy[0]],
            'top2_accuracy': [top10accuracy[1]],
            'top3_accuracy': [top10accuracy[2]],
            'top4_accuracy': [top10accuracy[3]],
            'top5_accuracy': [top10accuracy[4]],
            'top6_accuracy': [top10accuracy[5]],
            'top7_accuracy': [top10accuracy[6]],
            'top8_accuracy': [top10accuracy[7]],
            'top9_accuracy': [top10accuracy[8]],
            'top10_accuracy': [top10accuracy[9]],
            'optional_feature': [optional_feature],
            'learning_rate': [learning_rate],
            'dataset': [dataset],
            'experiment_num':[experiment_num],
    }

        # 创建DataFrame
    df = pd.DataFrame(data)

        # 检查train.csv文件是否存在来决定是否添加表头
    file_exists = os.path.isfile('modified_train_with_updated_duration.csv')

        # 如果文件存在，不写入表头，模式为追加；如果文件不存在，写入表头，模式为写入
    df.to_csv('modified_train_with_updated_duration.csv', mode='a', header=not file_exists, index=False)

    print(f'Epoch {epoch + 1} training data inserted into train.csv.')

Epoch 1:   0%|          | 0/36373 [00:00<?, ?it/s]

Validating:   0%|          | 0/6419 [00:00<?, ?it/s]

Accuracy after epoch 1: Top1: 70.28%
[70.27574388534039]
Accuracy after epoch 1: Top2: 77.07%
[70.27574388534039, 77.06807914005297]
Accuracy after epoch 1: Top3: 79.41%
[70.27574388534039, 77.06807914005297, 79.41268110297554]
Accuracy after epoch 1: Top4: 81.04%
[70.27574388534039, 77.06807914005297, 79.41268110297554, 81.04066053902477]
Accuracy after epoch 1: Top5: 82.10%
[70.27574388534039, 77.06807914005297, 79.41268110297554, 81.04066053902477, 82.10001557875059]
Accuracy after epoch 1: Top6: 82.89%
[70.27574388534039, 77.06807914005297, 79.41268110297554, 81.04066053902477, 82.10001557875059, 82.89453185854495]
Accuracy after epoch 1: Top7: 83.59%
[70.27574388534039, 77.06807914005297, 79.41268110297554, 81.04066053902477, 82.10001557875059, 82.89453185854495, 83.58778625954199]
Accuracy after epoch 1: Top8: 84.03%
[70.27574388534039, 77.06807914005297, 79.41268110297554, 81.04066053902477, 82.10001557875059, 82.89453185854495, 83.58778625954199, 84.03178065119178]
Accuracy aft

Epoch 2:   0%|          | 0/36373 [00:00<?, ?it/s]

Validating:   0%|          | 0/6419 [00:00<?, ?it/s]

...

Epoch 5:   0%|          | 0/36373 [00:00<?, ?it/s]

Validating:   0%|          | 0/6419 [00:00<?, ?it/s]

Accuracy after epoch 5: Top1: 76.81%
[76.81102975541361]
Accuracy after epoch 5: Top2: 82.89%
[76.81102975541361, 82.89453185854495]
Accuracy after epoch 5: Top3: 85.43%
[76.81102975541361, 82.89453185854495, 85.42607882847796]
Accuracy after epoch 5: Top4: 86.90%
[76.81102975541361, 82.89453185854495, 85.42607882847796, 86.89827075868516]
Accuracy after epoch 5: Top5: 87.93%
[76.81102975541361, 82.89453185854495, 85.42607882847796, 86.89827075868516, 87.93425767253466]
Accuracy after epoch 5: Top6: 88.80%
[76.81102975541361, 82.89453185854495, 85.42607882847796, 86.89827075868516, 87.93425767253466, 88.79887832995794]
Accuracy after epoch 5: Top7: 89.43%
[76.81102975541361, 82.89453185854495, 85.42607882847796, 86.89827075868516, 87.93425767253466, 88.79887832995794, 89.42981772861816]
Accuracy after epoch 5: Top8: 89.97%
[76.81102975541361, 82.89453185854495, 85.42607882847796, 86.89827075868516, 87.93425767253466, 88.79887832995794, 89.42981772861816, 89.96728462377317]
Accuracy aft

Epoch 6:   0%|          | 0/36373 [00:00<?, ?it/s]

KeyboardInterrupt: 